# 멜론 시대별 top 100 크롤링

In [99]:
!pip install pypandoc

  Created wheel for pypandoc: filename=pypandoc-1.5-cp36-none-any.whl size=17037 sha256=76f24bd7f132ffdaa5d2958dcefe29ea2de18b75138b90ae6d08ba6f154c01b3
  Stored in directory: /root/.cache/pip/wheels/bb/7d/d6/2f9af55e800d37e42e546106bcbd36a86e24e725e303d17e04
Successfully built pypandoc


In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
import re

### summary :

* 멜론 시대별 차트에서 1990년대(1990년 ~ 1999년)와 2000년대(2000년 ~ 2009년)의 top100 노래의 정보를 크롤링
* 각 시대별 유사곡 추천을 진행해본다.

* 각 데이터셋을 합쳐 전체 곡에서 유사곡 추천을 진행해본다. 

### 한계 :

* 가사라는 텍스트 데이터만 이용하여 마음에 드는 결과는 얻지 못했다.
* 토큰화를 진행하지 않아 상대적으로 유사곡 선정이 잘되지 않았다. ->  해결(Mecab)
* 음악의 특성상 음역대, 목소리 등 음성 데이터도 이용할 수 있었다면 보다 좋은 결과를 얻을 수 있을 것이다.

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [ ]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 56.5MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-07-22 05:05:01--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?S

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
driver = webdriver.Chrome()

## 1990년대(1990 ~ 1999)

In [ ]:
lyrics_1990_list = [] # 가사 리스트
song_name_1990_list = [] # 노래이름 리스트
singer_name_1990_list = [] # 가수 리스트

In [ ]:
for year in range(1990, 2000) : # 1990년 ~ 1999년 노래 크롤링
    print("##############################################\n{}년도의 top 100 노래 목록 정보 크롤링 시작".format(year))
    for page in [1, 51] :
        url = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={}#params%5Bidx%5D={}'.format(year, page)
        driver.get(url)
        time.sleep(.5)
        if page == 1 :
            css = 50
        else :
            css = 100
        for i in range(50) :
            try :
                rank = driver.find_elements_by_css_selector("#lst{} > td:nth-child(2) > div > span".format(css))[i].text
                time.sleep(.5)
                driver.find_elements_by_css_selector('#lst{} > td:nth-child(4) > div > a > span'.format(css))[i].click()
                time.sleep(1)
                song_name = driver.find_element_by_css_selector("#downloadfrm > div > div > div.entry > div.info > div.song_name").text
                song_name_1990_list.append(song_name) # 노래 이름 추가
                try : 
                    driver.find_element_by_css_selector("#lyricArea > button > span").click() # 가사 더보기 버튼 클릭
                    time.sleep(.5)
                    singer_name = driver.find_element_by_css_selector('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)').text
                    lyrics = re.sub('\n', ' ', driver.find_element_by_css_selector('#d_video_summary').text)
                    singer_name_1990_list.append(singer_name) # 가수이름 추가
                    lyrics_1990_list.append(lyrics) # 가사 추가
                    time.sleep(.5)
                    print("{}위 / 가수 : {} / 노래 제목 : {} / 가사일부 : {}".format(rank, singer_name, song_name, lyrics[:10])) # 노래 제목, 가수 이름, 가사(10글자까지) 표시
                except :
                    lyrics_1990_list.append('가사 없음')
                    singer_name_1990_list.append('미상')
                    print("{}위 / 가수 : {} / 노래 제목 : {} / 가사없음".format(rank, singer_name, song_name)) # 노래 제목, 가수 이름, 가사(10글자까지) 표시
                driver.get(url)
                time.sleep(1)
            except :
                time.sleep(1)
                pass
    print("##############################################\n{}년도의 top 100 노래 목록 정보 크롤링 완료".format(year))

##############################################
1990년도의 top 100 노래 목록 정보 크롤링 시작
1위 / 가수 : 변진섭 / 노래 제목 : 희망사항 / 가사일부 : 청바지가 잘 어울리
2위 / 가수 : 김민우 / 노래 제목 : 사랑일뿐야 / 가사일부 : 나를 어떻게 생각하
3위 / 가수 : 햇빛촌 / 노래 제목 : 유리창엔 비 / 가사일부 : 낮부터 내린 비는 
4위 / 가수 : 김현식 / 노래 제목 : 비오는 날 수채화 / 가사일부 : 빗방울 떨어지는 그
5위 / 가수 : 이승철 / 노래 제목 : 마지막 콘서트 / 가사일부 : 지금 슬픈 내 모습
6위 / 가수 : 조정현 / 노래 제목 : 그 아픔까지 사랑한거야 / 가사일부 : 너를 처음 만난 날
7위 / 가수 : 봄여름가을겨울 / 노래 제목 : 어떤이의 꿈 / 가사일부 : 어떤 이는 꿈을 간
8위 / 가수 : 신해철 / 노래 제목 : 슬픈 표정 하지 말아요 / 가사일부 : 이 세상 살아가는 
9위 / 가수 : 민해경 / 노래 제목 : 보고싶은 얼굴 / 가사일부 : 내사랑 어디 쯤에 
10위 / 가수 : 노래를 찾는 사람들 / 노래 제목 : 사계 / 가사일부 : 빨간 꽃 노란 꽃 
11위 / 가수 : 김민우 / 노래 제목 : 입영열차안에서 / 가사일부 : 어색해진 짧은 머리
12위 / 가수 : 박학기 / 노래 제목 : 향기로운 추억 (응답하라 1988 삽입곡) / 가사일부 : 한줌 젖은 바람은 
13위 / 가수 : 이승환 / 노래 제목 : 텅 빈 마음 / 가사일부 : 미련없이 그대를 떠
14위 / 가수 : 김지연 / 노래 제목 : 찬바람이 불면 / 가사일부 : 찬바람이 불면 내가
15위 / 가수 : 김현식 / 노래 제목 : 넋두리 / 가사일부 : 쓸쓸한 거리에 나 
16위 / 가수 : 최성수 / 노래 제목 : 잊지 말아요 / 가사일부 : 이젠 모두 지나버린
17위 / 가수 : 이상은 / 노래 제목 : 사랑할꺼야 / 가사일부 : 우리 이제는 좋아하
18위 / 가수 :

50위 / 가수 : 신해철 / 노래 제목 : 안녕 / 가사일부 : 선물가게의 포장지 
51위 / 가수 : 최용준 / 노래 제목 : 거울이 되어 / 가사일부 : 나를 향해 울어버리
52위 / 가수 : 심신 / 노래 제목 : 오직 하나뿐인 그대 / 가사일부 : 그리움 두고서 가지
53위 / 가수 : 유익종 / 노래 제목 : 차창에 흐르는 이별 / 가사일부 : 그때 나를 바라보던
54위 / 가수 : 이문세 / 노래 제목 : 가을이 가도 / 가사일부 : 가을이 가도 그대 
55위 / 가수 : 권인하 / 노래 제목 : 계절이 음악처럼 흐를때 / 가사일부 : 너의 말없는 눈빛과
56위 / 가수 : 권인하 / 노래 제목 : 16년차이 / 가사없음
57위 / 가수 : 노래마을 / 노래 제목 : 우리의 노래가 이 그늘진 땅에 햇볕 한 줌 될수 있다면 / 가사일부 : 우리의 노래가 이 
58위 / 가수 : 심명훈 / 노래 제목 : 슬픈 우리사랑 / 가사일부 : 그 아름답던 기억을
59위 / 가수 : 송골매 / 노래 제목 : 모여라 / 가사일부 : 모여라 모여라 모여
60위 / 가수 : 하덕규 / 노래 제목 : 쉼 / 가사일부 : 당신의 곁에 있어요
61위 / 가수 : 손무현 / 노래 제목 : 제목 없는 시(詩) / 가사일부 : 안개짙은 거리를 홀
62위 / 가수 : 태진아 / 노래 제목 : 거울도 안보는 여자 / 가사일부 : 1. 사랑찾아 헤매
63위 / 가수 : 임백천 / 노래 제목 : 헤어짐이 이별은 아니듯 / 가사일부 : 너의 마지막 모습을
64위 / 가수 : 주현미 / 노래 제목 : 잠깐만 / 가사일부 : 잠깐만 잠깐만 그 
65위 / 가수 : 박학기 / 노래 제목 : 자꾸 서성이게 돼 / 가사일부 : 언제라도 다시 느낄
66위 / 가수 : 임재범 / 노래 제목 : 이 밤이 지나면 / 가사일부 : 불빛만이 가득한 이
67위 / 가수 : 임재범 / 노래 제목 : Overture/추억속의 재회/마도요 / 가사없음
68위 / 가수 : 임재범 / 노래 제목 : 예감으로 느낀 너

1위 / 가수 : 한동준 / 노래 제목 : 너를 사랑해 / 가사일부 : 아침이 오는 소리에
2위 / 가수 : 이승철 / 노래 제목 : 방황 / 가사일부 : 매일 신문에 나고 
3위 / 가수 : 신승훈 / 노래 제목 : 널 사랑하니까 / 가사일부 : 난 너를 잊을수가 
4위 / 가수 : 피노키오 / 노래 제목 : 다시만난 너에게 / 가사일부 : 내가 처음 보았던 
5위 / 가수 : 김수희 / 노래 제목 : 애모 / 가사일부 : 그대 가슴에 얼굴을
6위 / 가수 : 김현철 / 노래 제목 : 그대안의 블루 / 가사일부 : 난 난 눈을 감아요
7위 / 가수 : 노이즈 / 노래 제목 : 너에게 원한건 / 가사일부 : 오늘도 우린 함께 
8위 / 가수 : 김광석 / 노래 제목 : 나의 노래 / 가사일부 : 아무 것도 가진 것
9위 / 가수 : 잼 / 노래 제목 : 난 멈추지 않는다 / 가사일부 : 이제 모든걸 다시 
10위 / 가수 : 장현철 / 노래 제목 : 걸어서 하늘까지 / 가사일부 : 눈 내리는 밤은 언
11위 / 가수 : 신효범 / 노래 제목 : 언제나 그 자리에 / 가사일부 : 어두운 창밖 불빛 
12위 / 가수 : 김종서 / 노래 제목 : 그래도 이제는 / 가사일부 : 그 모든건 어린 내
13위 / 가수 : 박정운 / 노래 제목 : 먼 훗날 / 가사일부 : 잊으려 잊으려 해도
14위 / 가수 : 푸른하늘 / 노래 제목 : 자아도취 / 가사일부 : 못생긴 얼굴에 작은
15위 / 가수 : 윤종신 / 노래 제목 : 너의 결혼식 / 가사일부 : 몰랐었어 네가 그렇
16위 / 가수 : 서태지와 아이들 / 노래 제목 : 하여가(何如歌) / 가사일부 : 너에게 모든 걸 뺏
17위 / 가수 : 김민종 / 노래 제목 : 하늘 아래서 / 가사일부 : 나의 과거를 이제는
18위 / 가수 : 이무송 / 노래 제목 : 사는게 뭔지 / 가사일부 : 사랑이 무엇인지 아
19위 / 가수 : 015B / 노래 제목 : 신(新) 인류의 사랑 / 가사일부 : 맘에 안드는 그녀에
20위 / 가

54위 / 가수 : 코코 / 노래 제목 : 그리움으로 지는 너 / 가사일부 : 거친 숨소리 속에 
55위 / 가수 : Kiky / 노래 제목 : 다른 모습 / 가사일부 : 굳이 말을 하지않아
56위 / 가수 : 김원준 / 노래 제목 : 짧은 다짐 / 가사일부 : 이제는 너를 잊겠다
57위 / 가수 : 전일식 / 노래 제목 : 널 기다리며 / 가사일부 : 또 다른 세상을 내
58위 / 가수 : 김수희 / 노래 제목 : 애모 / 가사일부 : 그대 가슴에 얼굴을
59위 / 가수 : 최진영 / 노래 제목 : 사랑을 그대 품안에 / 가사일부 : 가만히 다가가 손 
60위 / 가수 : 유영진 / 노래 제목 : 그대의 향기 / 가사일부 : 아직도 난 그대 꿈
61위 / 가수 : 화이트 / 노래 제목 : W.H.I.T.E. / 가사일부 : 램프의 요정을 따라
62위 / 가수 : 더 블루 / 노래 제목 : 그대와 함께 / 가사일부 : 그대여 나의 두 눈
63위 / 가수 : 장철웅 / 노래 제목 : 서울, 이곳은 / 가사일부 : 아무래도 난 돌아가
64위 / 가수 : 최연제 / 노래 제목 : 너를 잊을 수 없어 / 가사일부 : 멀어지는 너의 모습
65위 / 가수 : 오월 / 노래 제목 : 종로에서 / 가사일부 : 그렇게 떠나야만 했
66위 / 가수 : 이문세 / 노래 제목 : 종원에게 (눈물 흘리지 말아요) / 가사일부 : 눈물 흘리지 말아요
67위 / 가수 : 김종서 / 노래 제목 : 겨울비 / 가사일부 : 겨울비처럼 슬픈 노
68위 / 가수 : 모노 / 노래 제목 : Paradise / 가사일부 : 왜 내게 그런 말을
69위 / 가수 : 서태지와 아이들 / 노래 제목 : 난 알아요 / 가사일부 : I once met
70위 / 가수 : 김건모 / 노래 제목 : 어떤 기다림 / 가사일부 : 어제와 같은 시간이
71위 / 가수 : 박미경 / 노래 제목 : 이유같지 않은 이유 / 가사일부 : 아무것도 필요없어 
72위 / 가수 : 김민기 / 노래 제목 : 친구에게 (Feat. 전

1위 / 가수 : 패닉 / 노래 제목 : 달팽이 / 가사일부 : 집에 오는 길은 때
2위 / 가수 : 안치환 / 노래 제목 : 내가 만일 / 가사일부 : 내가 만일 하늘이라
3위 / 가수 : 동물원 / 노래 제목 : 널 사랑하겠어 / 가사일부 : 내 뜨거운 입술이 
4위 / 가수 : 조관우 / 노래 제목 : 모래성 / 가사일부 : 나를 사랑한다면 이
5위 / 가수 : 안상수 / 노래 제목 : 영원히 내게 / 가사일부 : 이 넓은 세상에 날
6위 / 가수 : 신해철 / 노래 제목 : 아주 가끔은 / 가사일부 : 1. 대낮에 길을 
7위 / 가수 : 김민종 / 노래 제목 : 귀천도애 (歸天道哀) / 가사일부 : 난 이대로 떠나 갈
8위 / 가수 : 영턱스클럽 / 노래 제목 : 정 (위험한 이별) / 가사일부 : 다른여자 생긴거라면
9위 / 가수 : 윤종신 / 노래 제목 : 환생 / 가사일부 : 다시 태어난 것 같
10위 / 가수 : 클론 / 노래 제목 : 꿍따리 샤바라 / 가사일부 : 마음이 울적하고 답
11위 / 가수 : 최재훈 / 노래 제목 : 잊을수 없는 너 / 가사일부 : 이렇게 너 떠나가고
12위 / 가수 : 솔리드 / 노래 제목 : 넌 나의 처음이자 마지막이야! / 가사일부 : 우리사랑 지금부터 
13위 / 가수 : 김건모 / 노래 제목 : 스피드 / 가사일부 : 허구헌날 매일 매일
14위 / 가수 : DJ DOC / 노래 제목 : 여름이야기 (Sky Mix) / 가사일부 : 지나간 여름 하하 
15위 / 가수 : 더 클래식 / 노래 제목 : 여우야(女雨夜) / 가사일부 : 창 밖엔 서글픈 비
16위 / 가수 : 장혜진 / 노래 제목 : 완전한 사랑 / 가사일부 : 나 이제 모든걸 그
17위 / 가수 : 장혜진 / 노래 제목 : 지금 이대로 / 가사없음
18위 / 가수 : 녹색지대 / 노래 제목 : 준비 없는 이별 / 가사일부 : 지난시간 내곁에서 
19위 / 가수 : 김성재 / 노래 제목 : 말하자면 / 가사일부 : 말 하자면 너를 사
20위 / 가수

51위 / 가수 : 이현도 / 노래 제목 : 사랑해 (Hiphop Ver.) (Contain Sample Of) / 가사일부 : 너의 그림이 또 그
52위 / 가수 : 이정봉 / 노래 제목 : 그녀를 위해 / 가사일부 : 사랑한다면 그녈 위
53위 / 가수 : 유피 / 노래 제목 : 뿌요 뿌요 / 가사일부 : 1 2 3 4 5 
54위 / 가수 : 베이시스 / 노래 제목 : 좋은사람 있으면 소개시켜줘 / 가사일부 : 워워워워우워~ 워워
55위 / 가수 : H.O.T. / 노래 제목 : Candy / 가사일부 : 사실은 오늘 너와의
56위 / 가수 : 클론 / 노래 제목 : 도시 탈출 / 가사일부 : 한낮에 찌는 듯한 
57위 / 가수 : 구피 / 노래 제목 : 비련 / 가사일부 : 저 달빛이 내 창에
58위 / 가수 : 레드플러스 / 노래 제목 : 그녀는... / 가사일부 : 한 여자의 남편이라
59위 / 가수 : 부활 / 노래 제목 : Lonely Night / 가사일부 : 이런 시간엔 더 그
60위 / 가수 : 솔리드 / 노래 제목 : 끼리 끼리 / 가사일부 : 넌 혹시 알고 있니
61위 / 가수 : 안치환 / 노래 제목 : 영산강 / 가사일부 : 차라리 울어 볼꺼나
62위 / 가수 : 임상아 / 노래 제목 : 뮤지컬 / 가사일부 : 내 삶을 그냥 내 
63위 / 가수 : 지누션 / 노래 제목 : 말해줘 / 가사일부 : 나에게 말해줘 사실
64위 / 가수 : 임상아 / 노래 제목 : 저 바다가 날 막겠어 / 가사일부 : 저 바다가 내 길을
65위 / 가수 : 젝스키스 / 노래 제목 : 사나이 가는 길 (폼생폼사) / 가사일부 : 나 폼에 살고 죽고
66위 / 가수 : 터보 / 노래 제목 : Good Bye Yesterday / 가사일부 : 나나나 나 나나나나
67위 / 가수 : 듀스 / 노래 제목 : 사랑, 두려움 / 가사일부 : 너를 안을 때마다 
68위 / 가수 : 유리상자 / 노래 제목 : 순애보 / 가사일부 : 난 망설였는데 너무
69위 / 가수 :

99위 / 가수 : 이현도 / 노래 제목 : 미래(未來) / 가사일부 : Oh baby ca
100위 / 가수 : 신효범 / 노래 제목 : 세상은 / 가사일부 : 귀기울여 듣고 있다
##############################################
1998년도의 top 100 노래 목록 정보 크롤링 완료
##############################################
1999년도의 top 100 노래 목록 정보 크롤링 시작
1위 / 가수 : 조성모 / 노래 제목 : 슬픈 영혼식 (For Your Soul) / 가사일부 : 아마 오셨을텐데 우
2위 / 가수 : 이승철 / 노래 제목 : 오직 너뿐인 나를… / 가사일부 : 아직도 널 잊지 못
3위 / 가수 : 양파 / 노래 제목 : A`ddio / 가사일부 : 그대 뒤에서 항상 
4위 / 가수 : K2 김성면 / 노래 제목 : 그녀의 연인에게... #Story Ⅰ / 가사일부 : 알고 있나요 지금 
5위 / 가수 : 쿨 (COOL) / 노래 제목 : 미절 (Misery) / 가사일부 : 누가 좀 나를 도와
6위 / 가수 : 핑클 (Fin.K.L) / 노래 제목 : 영원한 사랑 / 가사일부 : 이젠 내 사랑이 되
7위 / 가수 : 김현정 / 노래 제목 : 되돌아온 이별 / 가사일부 : 내겐 이런일 없을줄
8위 / 가수 : 유승준 / 노래 제목 : 열정 / 가사일부 : 이제 나 내가 말하
9위 / 가수 : 조성모 / 노래 제목 : 불멸의 사랑 / 가사일부 : 나 가진것 없지만 
10위 / 가수 : 유리상자 / 노래 제목 : 신부에게 / 가사일부 : 새하얀 드레스 수줍
11위 / 가수 : 유승준 / 노래 제목 : Shut Up!! (Feat. 이윤정) / 가사일부 : 어디서 뭘 했는데 
12위 / 가수 : 듀스 / 노래 제목 : 여름안에서 / 가사일부 : 언제나 꿈꿔 온 순
13위 / 가수 : 김경호 / 노래 제목 : 비정(非情) / 가사일부 : 손가락 하나 내 뜻
14위 / 가수 

### 취합 데이터를 데이터 프레임 형태로 변환

In [ ]:
song_lyrics_1990_df = pd.DataFrame({'가수' : singer_name_1990_list, '노래제목' : song_name_1990_list, '가사' : lyrics_1990_list})

In [ ]:
song_lyrics_1990_df = song_lyrics_1990_df.loc[song_lyrics_1990_df.가사 != '가사 없음']

In [ ]:
song_lyrics_1990_df.가사 = [re.findall(r'[가-힣]+', x) for x in song_lyrics_1990_df.가사] # 가사 중에서 한글만 추출

In [ ]:
song_lyrics_1990_df.가사 = [' '.join(x) for x in song_lyrics_1990_df.가사] # 리스트 형태가 개별 로우의 값으로 들어가 텍스트만 살림

In [ ]:
song_lyrics_1990_df['text'] = song_lyrics_1990_df.sum(axis = 1) # 중복된 곡들을 제거하기 위해 전체 텍스트 sum

In [ ]:
song_lyrics_1990_df['가수&제목'] = song_lyrics_1990_df.가수 + ' ' + song_lyrics_1990_df.노래제목 # 제목은 같지만 가사가 다른 노래를 구분하기 위해 "가수&제목" 구분자 생성

In [ ]:
song_lyrics_1990_df = song_lyrics_1990_df.drop_duplicates('text')
song_lyrics_1990_df.index = range(song_lyrics_1990_df.shape[0]) # 다음 과정에서 인덱싱 오류 방지를 위해 인덱스 재설정

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # 빈도수 기반 텍스트 마이닝을 위해 Tfidf 활용

In [ ]:
vectorizer = TfidfVectorizer(max_features = 750) # 최다 빈출 400개 단어 사용
tfidf = vectorizer.fit_transform(song_lyrics_1990_df.가사)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel # 텍스트간 유사도를 구하기 위해 코사인 유사도 사용
cosine_sim = linear_kernel(tfidf, tfidf)

In [ ]:
indices = pd.Series(song_lyrics_1990_df.index, index = song_lyrics_1990_df['가수&제목']).drop_duplicates()

### 유사곡 리스트(top-5) 출력 함수

In [ ]:
def get_similar_song(df, title, cosine_sim = cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6] # 자기자신(코사인 유사도 = 1)을 제외한 top5 노래 추출

    song_indices = [i[0] for i in sim_scores] # 추출된 top5 노래 인덱스 저장
    print('입력한 노래 제목은 "{}"의 "{}"'.format(df.가수[idx], df.노래제목[idx])) # 입력한 노래의 가수와 제목 표시
    return  display(df[['가수', '노래제목']].iloc[song_indices]) # 유사한 노래 5곡의 가수와 제목을 테이블 형태로 반환

In [ ]:
get_similar_song(song_lyrics_1990_df, song_lyrics_1990_df['가수&제목'][750])

입력한 노래 제목은 "김경호"의 "금지된 사랑"


,가수,노래제목
515,R.ef,찬란한 사랑 (상심II)
297,김명상,거기 지금 누구인가
778,박지윤,가버려!!
284,정연준,Pilot
351,낯선 사람들,낯선 사람들


In [ ]:
song_lyrics_1990_df.가사[750]

'울지 마 여기에 새겨진 우리 이름을 봐 소중한 초대장이 젖어버리잖아 슬퍼마 너의 가족들이 보이지 않아도 언젠간 용서할 그 날이 올 거야 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속 해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 각오해 내게 무릎 꿇은 세상의 복수를 많은 시련 준대도 널 위해 견딜게 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 긴 세월 흐른 뒤 돌아보아도 아무런 후회 없도록 단 하룰 살아도 너 행복 하도록 만들 거야 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 사랑할게'

In [ ]:
song_lyrics_1990_df.가사[515]

'너에게 아무런 도움이 되지 못했어 내가 내 불행마저 감당할 수 없는데 그냥 너를 좋아한 시작부터가 잘못이었다고 후회하고 있을 뿐이야 그래서 떠나는거야 잠시 먼 여행을 떠나 너에게 돌아오고 싶은 내 의지마저 억누르면서 그냥 널 잊어보는거야 한참 후 네가 날 잊을때 쯤에 난 너의 등뒤에 편지를 쓰겠지 부칠수도 읽을수도 없는 눈물로쓴 편지들을 먼 훗날 내 죽음 같은 사랑을 느꼈을 때 내이름을 불러줘 얼마 살지 못할 내 기억의 생명을 다시 살릴 수 있게 수북한 나의 그리움들 담을 수 없게 된 작은 마음을 탓하지마라 날 떠날 수 밖에 없는 내 마음을 날 보내주겠니 너 하나만으로 아름다웠던 수 많은 기억속으로 가서 날 찾아와도 널 껴안아 줄 수 없어 난 힘이 없어 슬픔보다도 더 깊은 추억이 있어 너에게 다시 만날 약속이 없어도 언제나 눈물은 짓지마 한번 더 한번 더 사랑이란 쓴약에 빠져들잖아 하지만 하고 싶다면 다 이룰 수 없는 사랑에 너는 쓰러지지 않도록 찬란한 사랑에 눈이 멀어야 하지 멀 그렇게 생각하는 거니 나의 불행마저 사랑할 수 있다는 것은 나의 짐이 무거워 지는거야 생각해봐 추억은 길들여진 환상일 뿐 현실 속에 나를 이해하고 이젠 멀리멀리 떠나 줘 멀리 떠나줘 슬픔보다도 더 깊은 추억이 있어 너에게 다시 만날 약속이 없어도 언제나 눈물은 짓지마 한번 더 한번 더 사랑이란 쓴약에 빠져들잖아 하지만 하고 싶다면 다 이룰 수 없는 사랑에 너는 쓰러지지 않도록 찬란한 사랑에 눈이 멀어야 하지 너는 쓰러지지 않도록 찬란한 사랑에 눈이 멀어야 하지'

In [ ]:
get_similar_song(song_lyrics_1990_df, song_lyrics_1990_df['가수&제목'][680])

입력한 노래 제목은 "S.E.S."의 "(`Cause) I`m Your Girl"


,가수,노래제목
485,솔리드,넌 나의 처음이자 마지막이야!
757,이승철,오직 너뿐인 나를…
733,엔알지 (NRG),사랑만들기
313,부활,사랑할수록
441,컬트,너를 품에 안으면


In [ ]:
song_lyrics_1990_df['가사'][680]

'왜 내게 말을 못해 이미 지나간 일들 진부한 옛 사랑 얘기 솔직히 말을 해줘 그렇지만 너에겐 오직 나뿐인 거야 두려워 하지마 내 곁에 있는걸 그대와 내 인생 저 끝까지 나를 믿어주길 바래 함께있어 너를 닮아가는 내 모습을 지켜봐줘 넌 왠지 달랐었지 느낌이 예전부터 알고 지낸 친구처럼 그렇게 너는 내게 해맑은 웃음만을 주는 또 하나의 나 두려워 하지마 네 옆에 있는걸 난 오직 너를 위해 살고 싶어 나를 향한 네 모든걸 간직할께 너를 닮아가는 내 모습 지켜봐줘 몰랐어 난 나 너에게 있어 그 어느 하나 자신있게 얘기할 수 없었던 것 모두 부질없는 내 걱정이었다니 이제 널 나의 품에 꼭 안아줄게 난 널 몰랐던거야 오직 너 하나 하나를 위해 내가 살아가야 한다는 것말고 내게 중요한 게 어딨겠어 이젠 네가 알아주길 바랬어 너에겐 그 어떤 말보다 넌 내꺼라는 말이 듣고 싶어 사랑해 언제까지나 나를 믿어주길 바래 함께있어 너를 닮아가는 내 모습 지켜봐줘 나를 향한 네 모든걸 간직할께 너를 닮아가는 내 모습 지켜봐줘'

In [ ]:
song_lyrics_1990_df['가사'][485]

'우리사랑 지금부터 시작인거야 아무 말도 하지마 어둠 만이 너와 나를 만질 수 있어 내 가슴에 기대봐 들리지 않니 너를 향한 나의 마음이 누구 하나 축복해 줄 사람 없는 시간 속에 더 이상은 가슴조일 필요는 없어 내 삶 속의 방황은 모두 끝났어 너를 만나기 전의 가슴앓이 뿐 우리 사랑 지금부터 시작인거야 그 누구도 간섭할 수 없는 얘기지 너를 위해 무엇이든 난 할 수 있어 넌 나의 모든 것 처음이자 마지막이야 눈을 뜨려 하지마 느낌으로 너와 나를 확인하면 돼 나의 품에 안겨봐 들리지 않니 사랑한단 나의 그 말이 어느 순간 떠올라서 저 하늘을 날 것 같아 이젠 정말 그 무엇도 두렵지 않아 내 삶 속의 방황은 모두 끝났어 너를 만나기 전의 가슴앓이뿐 우리 사랑 지금부터 시작인거야 그 누구도 간섭할 수 없는 얘기지 너를 위해 무엇이든 난 할 수 있어 넌 나의 모든 것 처음이자 마지막이야 아무 말도 하지마 어둠 만이 너와 나를 만질 수 있어 내 가슴에 기대봐 들리지 않니 너를 향한 나의 마음이 내 모든걸 주고 싶어 넌 나의 처음이자 마지막이야 너는 나의 그 모든 것 처음이자 마지막이야'

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA # 토픽분석을 위해 LDA 사용

In [ ]:
lda_model = LDA(n_components = 3, random_state = 721, max_iter = 2) # 최대 3개 주제 추출

In [ ]:
lda_top = lda_model.fit_transform(tfidf) # Tfidf 결과물을 lda로 변환

In [ ]:
terms = vectorizer.get_feature_names() # 위에서 지정한 max_features인 750개의 단어가 저장됨

def get_topics(components, feature_names, n = 5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('그대', 16.73), ('나를', 12.89), ('내가', 12.74), ('내게', 11.21), ('나의', 11.19)]
Topic 2: [('너를', 17.93), ('나의', 16.96), ('나를', 15.23), ('너의', 11.87), ('다시', 11.62)]
Topic 3: [('너의', 14.82), ('너를', 12.94), ('그대', 12.76), ('내가', 11.67), ('나의', 10.9)]


### 토큰화를 하지 않아서 중복되는 의미의 단어들이 토픽에 영향을 끼치는 단어로 선정되었다.

In [ ]:
song_lyrics_1990_df.to_csv('1990_2000_song.csv', index = False, encoding = 'utf-8')

## 2000년대(2000 ~ 2009)

In [ ]:
lyrics_2000_list = [] # 가사 리스트
song_name_2000_list = [] # 노래이름 리스트
singer_name_2000_list = [] # 가수 리스트

In [ ]:
for year in range(2000, 2010) : # 2000년 ~ 2009년 노래 크롤링
    print("##############################################\n{}년도의 top 100 노래 목록 정보 크롤링 시작".format(year))
    for page in [1, 51] :
        url = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={}#params%5Bidx%5D={}'.format(year, page)
        driver.get(url)
        time.sleep(.5)
        if page == 1 :
            css = 50
        else :
            css = 100
        for i in range(50) :
            rank = driver.find_elements_by_css_selector("#lst{} > td:nth-child(2) > div > span".format(css))[i].text
            time.sleep(.5)
            try :
                driver.find_elements_by_css_selector('#lst{} > td:nth-child(4) > div > a > span'.format(css))[i].click()
                time.sleep(1)
                song_name = driver.find_element_by_css_selector("#downloadfrm > div > div > div.entry > div.info > div.song_name").text
                song_name_2000_list.append(song_name) # 노래 이름 추가

                try : 
                    driver.find_element_by_css_selector("#lyricArea > button > span").click() # 가사 더보기 버튼 클릭
                    time.sleep(.5)
                    singer_name = driver.find_element_by_css_selector('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)').text
                    lyrics = re.sub('\n', ' ', driver.find_element_by_css_selector('#d_video_summary').text)
                    singer_name_2000_list.append(singer_name) # 가수이름 추가
                    lyrics_2000_list.append(lyrics) # 가사 추가
                    time.sleep(.5)
                    print("{}위 / 가수 : {} / 노래 제목 : {} / 가사일부 : {}".format(rank, singer_name, song_name, lyrics[:10])) # 노래 제목, 가수 이름, 가사(10글자까지) 표시
                except :
                    lyrics_2000_list.append('가사 없음')
                    singer_name_2000_list.append('미상')
                    print("{}위 / 가수 : {} / 노래 제목 : {} / 가사없음".format(rank, singer_name, song_name)) # 노래 제목, 가수 이름, 가사(10글자까지) 표시
                driver.get(url)
                time.sleep(1)
            except :
                time.sleep(1)
                pass
    print("##############################################\n{}년도의 top 100 노래 목록 정보 크롤링 완료".format(year))

##############################################
2000년도의 top 100 노래 목록 정보 크롤링 시작
1위 / 가수 : 조성모 / 노래 제목 : 아시나요 / 가사일부 : 아시나요 얼마나 사
2위 / 가수 : 조규만 / 노래 제목 : 다 줄거야 (Acoustic Ver.) / 가사일부 : 그대 내게 다가오는
3위 / 가수 : DJ DOC / 노래 제목 : Run To You / 가사일부 : 하나 투 원 투 쓰
4위 / 가수 : god / 노래 제목 : 거짓말 / 가사일부 : 미안해 난 니가 싫
5위 / 가수 : 클론 / 노래 제목 : 초련(初戀) (Techno Mix) (Feat. 윤진) / 가사일부 : 난 그냥 좋았어 니
6위 / 가수 : 조성모 / 노래 제목 : 가시나무 / 가사일부 : 내 속엔 내가 너무
7위 / 가수 : 홍경민 / 노래 제목 : 흔들린 우정 / 가사일부 : 아냐 이게 아닌데 
8위 / 가수 : 임창정 / 노래 제목 : 나의 연인(我戀) / 가사일부 : 너를 알기 전에 나
9위 / 가수 : 스카이 / 노래 제목 : 영원 / 가사일부 : 너와의 행복했었던 
10위 / 가수 : 김현정 / 노래 제목 : 멍 / 가사일부 : 너 나를 쉽게 봤어
11위 / 가수 : 컨츄리 꼬꼬 / 노래 제목 : 오! 가니 / 가사일부 : 오오 오오 우리 오
12위 / 가수 : 정일영 / 노래 제목 : 기도 / 가사일부 : 혹시 나를 떠나려는
13위 / 가수 : 신승훈 / 노래 제목 : 전설속의 누군가처럼 / 가사일부 : 거울 속의 그대 모
14위 / 가수 : 임재범 / 노래 제목 : 너를 위해 / 가사일부 : 어쩜 우린 복잡한 
15위 / 가수 : 박효신 / 노래 제목 : 바보 / 가사일부 : 너무 모르고 있죠 
16위 / 가수 : 이승환 / 노래 제목 : 그대가 그대를 / 가사일부 : 참 많이 모질었었죠
17위 / 가수 : 자우림 / 노래 제목 : 매직 카펫 라이드 / 가사일부 : 이렇게 멋진 파란 
18위 / 가수 : 조장

48위 / 가수 : S.E.S. / 노래 제목 : 감싸 안으며 (Show Me Your Love) / 가사일부 : 비 개인 거리를 나
49위 / 가수 : 박효신 / 노래 제목 : 동경 / 가사일부 : 우린 서로 너무도 
50위 / 가수 : YDG / 노래 제목 : 구리뱅뱅 (Feat. 1kyne, 난아) / 가사일부 : Just anoth
51위 / 가수 : 브라운 아이즈 / 노래 제목 : With Coffee... / 가사일부 : Cause you'
52위 / 가수 : 신승훈 / 노래 제목 : I Believe / 가사일부 : I believe 
53위 / 가수 : 차태현 / 노래 제목 : I Love You / 가사일부 : 후회해 내 사랑을 
54위 / 가수 : 제이 / 노래 제목 : 빛 / 가사일부 : taking it 
55위 / 가수 : 박효신 / 노래 제목 : 바보 / 가사일부 : 너무 모르고 있죠 
56위 / 가수 : 강타 (KANGTA) / 노래 제목 : 북극성 / 가사일부 : 괜찮은 건지 혹시 
57위 / 가수 : 조성모 / 노래 제목 : 잘가요 내사랑 / 가사일부 : 그죠 날 떠나실거죠
58위 / 가수 : 홍경민 / 노래 제목 : 가져가 / 가사일부 : 뭐하는 거야 지금 
59위 / 가수 : 유승준 / 노래 제목 : Wow (Ver.2) / 가사일부 : wow 예감이 괜찮
60위 / 가수 : 김장훈 / 노래 제목 : 난 남자다 / 가사일부 : 넌 모르겠지만 사랑
61위 / 가수 : 유엔 / 노래 제목 : 파도 / 가사일부 : 눈이 부시게 아름답
62위 / 가수 : 강성훈 / 노래 제목 : 축복 / 가사일부 : 알아 이제는 오지 
63위 / 가수 : 싸이 (PSY) / 노래 제목 : 새 / 가사일부 : 당신 너무나 이쁜 
64위 / 가수 : 샵 / 노래 제목 : Sweety / 가사일부 : 퐁당퐁당 너를 처음
65위 / 가수 : 이브 / 노래 제목 : I` ll Be There / 가사일부 : I'm so lon
66위 / 가수 : 플라이 투 

95위 / 가수 : 제이워크 / 노래 제목 : Suddenly / 가사일부 : Suddenly 이
96위 / 가수 : 샵 / 노래 제목 : 눈물 / 가사일부 : 오렐레 오랄라 he
97위 / 가수 : 윤상 / 노래 제목 : 이사(移徙) / 가사일부 : 이젠 출발이라고 문
98위 / 가수 : 강타 (KANGTA) / 노래 제목 : 사랑은 기억보다 (Memories) / 가사일부 : 한참을 말이 없이 
99위 / 가수 : 악동클럽 / 노래 제목 : Remember / 가사일부 : tell me wh
100위 / 가수 : JK 김동욱 / 노래 제목 : 그녈 위해 / 가사일부 : 비가 오는 거릴 좋
##############################################
2002년도의 top 100 노래 목록 정보 크롤링 완료
##############################################
2003년도의 top 100 노래 목록 정보 크롤링 시작
1위 / 가수 : 빅마마 / 노래 제목 : Break Away / 가사일부 : 이젠 날 떠나가줘 
2위 / 가수 : 브라운 아이즈 / 노래 제목 : 점점 / 가사일부 : 점점 넌 멀어지나봐
3위 / 가수 : 휘성 (Realslow) / 노래 제목 : With Me / 가사일부 : This song 
4위 / 가수 : 브라운 아이드 소울 / 노래 제목 : 정말 사랑했을까 / 가사일부 : 좋았나봐 널 많이 
5위 / 가수 : 보아 (BoA) / 노래 제목 : 아틀란티스 소녀 (Atlantis Princess) / 가사일부 : 아틀란티스 소녀 (
6위 / 가수 : 쿨 (COOL) / 노래 제목 : 결혼을 할 거라면 / 가사일부 : 이리저리 아무리 생
7위 / 가수 : 이효리 / 노래 제목 : 10 Minutes / 가사일부 : Just One 1
8위 / 가수 : YB / 노래 제목 : 잊을께 / 가사일부 : 아침에 눈을 떴을 
9위 / 가수 : 윤건 / 노래 제목 : 어쩌다.. / 가사일부 : 어쩌

34위 / 가수 : 이승철 / 노래 제목 : 인연 / 가사일부 : 눈을 떠 바라보아요
35위 / 가수 : 엠씨더맥스 (M.C the MAX) / 노래 제목 : 그대는 눈물겹다 / 가사일부 : 나에게 곧 올거라고
36위 / 가수 : 김연우 / 노래 제목 : 연인 / 가사일부 : 언제 보여줄꺼야 니
37위 / 가수 : 다이나믹 듀오 / 노래 제목 : Ring My Bell (Feat. 나얼 Of Brown Eyed Soul) / 가사일부 : you 내가 필요할
38위 / 가수 : V.One / 노래 제목 : 그런가 봐요... / 가사일부 : 고맙다는 그 말 잘
39위 / 가수 : 버즈 / 노래 제목 : Monologue / 가사일부 : 다 잊었다는 거짓말
40위 / 가수 : 신승훈 / 노래 제목 : 그런 날이 오겠죠 / 가사일부 : 아픈 사람같아 보인
41위 / 가수 : 럼블피쉬 / 노래 제목 : 예감 좋은 날 / 가사일부 : 한 동안 내리던 비
42위 / 가수 : 이승기 / 노래 제목 : 내 여자라니까 / 가사일부 : 나를 동생으로만 그
43위 / 가수 : 태빈 / 노래 제목 : 내가 눈을 감는 이유 / 가사일부 : 언제부터였는지 자꾸
44위 / 가수 : 김종국 / 노래 제목 : 한 남자 / 가사일부 : 참 오래됐나봐 이 
45위 / 가수 : 서태지 / 노래 제목 : 로보트 / 가사일부 : 매년 내 방문 기둥
46위 / 가수 : 러브홀릭 / 노래 제목 : Sky / 가사일부 : 이미 너는 너무 먼
47위 / 가수 : 주석 / 노래 제목 : 정상을 향한 독주 2 (Feat. 김범수) / 가사일부 : 끝없는 설레임을 넘
48위 / 가수 : 성시경 / 노래 제목 : 제주도의 푸른 밤 / 가사일부 : 떠나요 둘이서 모든
49위 / 가수 : 장나라 / 노래 제목 : 그게 정말이니? / 가사일부 : 후회는 없을거라고 
50위 / 가수 : 이수영 / 노래 제목 : 꿈에 / 가사일부 : 꿈에 어제꿈에 보았
51위 / 가수 : 테이 / 노래 제목 : 닮은 사람 / 가사일부 : 

76위 / 가수 : 쿨 (COOL) / 노래 제목 : 이여름 Summer / 가사일부 : 너무나 소중했던 사
77위 / 가수 : god / 노래 제목 : 2♡ / 가사일부 : I’m sorry 
78위 / 가수 : 유엔 / 노래 제목 : 그녀에게 / 가사일부 : 뛰어가는 그녀에게 
79위 / 가수 : 신화 / 노래 제목 : 세월의 흔적 다 버리고 / 가사일부 : 헤어졌어도 시간이 
80위 / 가수 : 이수영 / 노래 제목 : 꽃들은 지고 / 가사일부 : 그래도 내 곁을 떠
81위 / 가수 : 버즈 / 노래 제목 : 1st / 가사일부 : 시간이 흘러가도 변
82위 / 가수 : 서지영 / 노래 제목 : Stay in Me / 가사일부 : 너만을 원해도 사랑
83위 / 가수 : 코요태 / 노래 제목 : 빙고 / 가사일부 : Yo How you
84위 / 가수 : 거미 / 노래 제목 : 혼자만 하는 사랑 / 가사일부 : 한번만 날 봐달라고
85위 / 가수 : SG 워너비 / 노래 제목 : 꿈의 대화 (R&B Soul ver.) / 가사일부 : 땅거미 내려앉아 어
86위 / 가수 : 가비엔제이 / 노래 제목 : 그래도 살아가겠지 / 가사일부 : 널 잊을만큼 멀리왔
87위 / 가수 : 동방신기 (TVXQ!) / 노래 제목 : Rising Sun (순수) / 가사일부 : Now I cry 
88위 / 가수 : 박상민 / 노래 제목 : 눈물잔 / 가사일부 : 나처럼 말하고 나처
89위 / 가수 : 동방신기 (TVXQ!) / 노래 제목 : Hi Ya Ya 여름날 / 가사일부 : Beautiful 
90위 / 가수 : 이소은 / 노래 제목 : 닮았잖아 / 가사일부 : 꿈속에서 니 손을 
91위 / 가수 : 엠씨더맥스 (M.C the MAX) / 노래 제목 : 사랑은 아프려고 하는 거죠 / 가사일부 : 눈부신 햇살이 밤하
92위 / 가수 : 홍경민 / 노래 제목 : Tonight / 가사일부 : 아름다운 그녀가 워
93위 / 가수 : 나얼 / 노래 제목 : 언젠가는 / 가사일부

14위 / 가수 : 에픽하이 (EPIK HIGH) / 노래 제목 : Fan / 가사일부 : oh 가질수가 없어
15위 / 가수 : BIGBANG / 노래 제목 : 거짓말 / 가사일부 : ye love is
16위 / 가수 : 케이윌 / 노래 제목 : 왼쪽가슴 / 가사일부 : 그렇게 쉬울 줄 몰
17위 / 가수 : 먼데이 키즈 (Monday Kiz) / 노래 제목 : 남자야 / 가사일부 : 이미 다른 사람 곁
18위 / 가수 : SG 워너비 / 노래 제목 : 한여름날의 꿈 (Feat. 옥주현) / 가사일부 : 나의 꿈이 하나 있
19위 / 가수 : EVAN 에반 (유호석) / 노래 제목 : 남자도... 어쩔 수 없다. / 가사일부 : 입술이 모두 말라가
20위 / 가수 : 최현준 (V.O.S) / 노래 제목 : 나 이젠 (Duet 진성 Of 먼데이키즈) / 가사일부 : 아침에 일어나 눈을
21위 / 가수 : 엠씨더맥스 (M.C the MAX) / 노래 제목 : 가슴아 그만해 / 가사일부 : 참 많이 사랑했다는
22위 / 가수 : FTISLAND (FT아일랜드) / 노래 제목 : 사랑앓이 / 가사일부 : 그리울 때 눈 감으
23위 / 가수 : 양파 / 노래 제목 : 사랑..그게 뭔데 / 가사일부 : 무슨 뜻인 건가요 
24위 / 가수 : 씨야 / 노래 제목 : 사랑의 인사 / 가사일부 : 비내리는 거릴 좋아
25위 / 가수 : 원더걸스 / 노래 제목 : Irony (아이러니) / 가사일부 : 자꾸 첨이란 말을 
26위 / 가수 : 원더걸스 / 노래 제목 : Tell me (Sampling From `Two Of Hearts`) / 가사일부 : 너도 날 좋아할 줄
27위 / 가수 : 이효리 / 노래 제목 : 톡!톡!톡! (Toc Toc Toc)(CF - 투싼 `이효리`편) / 가사일부 : Yeah turn 
28위 / 가수 : 마골피 / 노래 제목 : 비행소녀 / 가사일부 : 활주로를 떠나 비행
29위 / 가수 : MC몽 / 노래 제목 : So Fresh (Feat

47위 / 가수 : 비 / 노래 제목 : Love Story / 가사일부 : Hey this i
48위 / 가수 : 이진성 (먼데이 키즈) / 노래 제목 : 그깟 사랑 / 가사일부 : 아직 모르겠니 모든
49위 / 가수 : 에픽하이 (EPIK HIGH) / 노래 제목 : 1분 1초 (Feat. 타루) / 가사일부 : I can't le
50위 / 가수 : 김종국 / 노래 제목 : 어제보다 오늘 더 / 가사일부 : I do 영원히 단
51위 / 가수 : MC몽 / 노래 제목 : 죽도록 사랑해 (Feat. 박정현) / 가사일부 : 죽도록 그대를 죽도
52위 / 가수 : 브라운아이드걸스 / 노래 제목 : You / 가사일부 : yeah yeah 
53위 / 가수 : 이승기 / 노래 제목 : 다 줄꺼야 / 가사일부 : 그대 내게 다가오는
54위 / 가수 : 알렉스 / 노래 제목 : 화분 (알렉스 Ver.) (우리결혼했어요 삽입곡) / 가사일부 : 멀리서 멀리서 멀리
55위 / 가수 : 비 / 노래 제목 : Rainism (Clean Ver.) / 가사일부 : I'm gonna 
56위 / 가수 : 원더걸스 / 노래 제목 : 이바보 / 가사일부 : 친구들이 내게 니가
57위 / 가수 : BIGBANG / 노래 제목 : 붉은 노을 / 가사일부 : 난 너를 사랑해 L
58위 / 가수 : BIGBANG / 노래 제목 : 착한 사람 / 가사일부 : 착한 사람이였음 안
59위 / 가수 : 환희 / 노래 제목 : 내사람 / 가사일부 : 난 웃어도 눈물이 
60위 / 가수 : 2PM / 노래 제목 : 10점 만점에 10점 / 가사일부 : That girl 
61위 / 가수 : 엄정화 / 노래 제목 : D.I.S.C.O (Feat. TOP) / 가사일부 : D I S C O 
62위 / 가수 : 성시경 / 노래 제목 : 안녕 나의 사랑 / 가사일부 : 여름 냄새 벌써 이
63위 / 가수 : 노블레스 / 노래 제목 : 후회는 없어 (Feat. 베이지) / 가사일부 : 사랑이 떠나가네 

82위 / 가수 : 왁스 / 노래 제목 : 전화 한 번 못하니 (Feat. 미스에스 태혜영) / 가사일부 : 전화 한 번 못하니
83위 / 가수 : BIGBANG / 노래 제목 : Wonderful / 가사일부 : 도대체 뭐가 그리 
84위 / 가수 : 케이윌 / 노래 제목 : 그립고 그립고 그립다 / 가사일부 : 그립고 그립고 그립
85위 / 가수 : G-DRAGON / 노래 제목 : 소년이여 / 가사일부 : GD A yo ch
86위 / 가수 : 퓨처라이거 / 노래 제목 : Let`s Dance / 가사일부 : 우리와 함께 하시겠
87위 / 가수 : 김용준 / 노래 제목 : 커플 (Feat. 황정음) / 가사일부 : 이젠 고백할래 ba
88위 / 가수 : 2PM / 노래 제목 : 기다리다 지친다 / 가사일부 : 기다리다 지친다 U
89위 / 가수 : 프리스타일 / 노래 제목 : 마음으로 하는 말 (Feat. Hanyi) / 가사일부 : 그대없이 보낸하루는
90위 / 가수 : f(x) / 노래 제목 : Chu~♡ / 가사일부 : Do it do i
91위 / 가수 : BIGBANG / 노래 제목 : Stylish (The FILA) / 가사일부 : ma energy 
92위 / 가수 : MC몽 / 노래 제목 : Simple Love (Feat. 나비) / 가사일부 : 분노 두려움 슬픔 
93위 / 가수 : 채연 / 노래 제목 : 흔들려 / 가사일부 : 언제나 너는 날 편
94위 / 가수 : 다이나믹 듀오 / 노래 제목 : 죽일 놈 (Guilty) / 가사일부 : 너 아까 나한테 왜
95위 / 가수 : 부활 / 노래 제목 : 생각이나 / 가사일부 : 항상 난 생각이나 
96위 / 가수 : 손담비 / 노래 제목 : 아몰레드 (Amoled) / 가사일부 : 빛나지 그 누구보다
97위 / 가수 : 김종국 / 노래 제목 : 행복병 (Feat. Shorry J (MM)) / 가사일부 : 어떻게 내가 널 찾
98위 / 가수 : 마이티 마우스 / 노래 제목 : 연애특강 (

### 취합 데이터를 데이터 프레임 형태로 변환

In [ ]:
song_lyrics_df_2000 = pd.DataFrame({'가수' : singer_name_2000_list, '노래제목' : song_name_2000_list, '가사' : lyrics_2000_list})

In [ ]:
song_lyrics_df_2000 = song_lyrics_df_2000.loc[song_lyrics_df_2000.가사 != '가사 없음']

In [ ]:
song_lyrics_df_2000.head()

,가수,노래제목,가사
0,조성모,아시나요,아시나요 얼마나 사랑했는지 그댈 보면 자꾸 눈물이 나서 차마 그대 바라보지 못하고 ...
1,조규만,다 줄거야 (Acoustic Ver.),그대 내게 다가오는 그 모습 자꾸 다시 볼수 없을 것만 같아서 감은 두눈 뜨지 못한...
2,DJ DOC,Run To You,하나 투 원 투 쓰리 포 BOUNCE with me BOUNCE with me BO...
3,god,거짓말,미안해 난 니가 싫어졌어 우리 이만 헤어져 다른 여자가 생겼어 너보다 훨씬 좋은 실...
4,클론,초련(初戀) (Techno Mix) (Feat. 윤진),난 그냥 좋았어 니 앞에만 서면 요즘 내가 왜 이러는지 나도 몰라 니 생각만 하고 ...


In [ ]:
song_lyrics_df_2000.가사 = [re.findall(r'[가-힣]+', x) for x in song_lyrics_df_2000.가사] # 가사 중에서 한글만 추출

In [ ]:
song_lyrics_df_2000.가사 = [' '.join(x) for x in song_lyrics_df_2000.가사] # 리스트 형태가 개별 로우의 값으로 들어가 텍스트만 살림

In [ ]:
song_lyrics_df_2000['text'] = song_lyrics_df_2000.sum(axis = 1) # 중복된 곡들을 제거하기 위해 전체 텍스트 sum

In [ ]:
song_lyrics_df_2000['가수&제목'] = song_lyrics_df_2000.가수 + ' ' + song_lyrics_df_2000.노래제목 # 제목은 같지만 가사가 다른 노래를 구분하기 위해 "가수&제목" 구분자 생성

In [ ]:
song_lyrics_df_2000 = song_lyrics_df_2000.drop_duplicates('text')
song_lyrics_df_2000.index = range(song_lyrics_df_2000.shape[0]) # 다음 과정에서 인덱싱 오류 방지를 위해 인덱스 재설정

### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features = 750) # 최다 빈출 750개 단어 사용
tfidf = vectorizer.fit_transform(song_lyrics_df_2000.가사)

In [ ]:
cosine_sim = linear_kernel(tfidf, tfidf)

In [ ]:
indices = pd.Series(song_lyrics_df_2000.index, index = song_lyrics_df_2000['가수&제목']).drop_duplicates()

### 유사곡 리스트(top 5) 출력 함수

In [ ]:
get_similar_song(song_lyrics_df_2000, song_lyrics_df_2000['가수&제목'][765])

입력한 노래 제목은 "이수 (엠씨더맥스)"의 "미련한 가슴아"


,가수,노래제목
450,이승환,심장병
675,에픽하이 (EPIK HIGH),Paris (Feat. Jisun of Loveholic)
489,강성훈,보이지 않는 인사
731,휘성 (Realslow),사랑은 맛있다♡
596,바이브,그 남자 그 여자 (Feat. 장혜진)


In [ ]:
song_lyrics_df_2000.가사[765]

'너와 함께한 시간 너만 사랑한 날들 나의 삶의 이유는 오직 너였어 이젠 널 보내야 해 너를 떠나가야 해 다시는 뒤돌아보지 않겠어 미련한 가슴아 아련한 추억아 그 사랑 아니라고 제발 내게 말해줘 두 눈이 멀어서 널 보지못하게 찢겨진 아픔만 안고 살아도 끝내 울음을 참고 다문 입술 깨물어 너완 상관없다고 모진 말해도 결국 그리워하며 내 눈 감는 날까지 널 보낸 오늘을 후회할 거야 미련한 가슴아 아련한 추억아 그 사랑 아니라고 제발 내게 말해줘 두 눈이 멀어서 널 보지못하게 찢겨진 아픔만 안고 살아도 널 아껴줄 좋은 사람 난 아니였나봐 미안해 미련한 가슴아 아련한 추억아 그 사랑 아니라고 제발 내게 말해줘 두 눈이 멀어서 널 보지못하게 찢겨진 아픔만 안고 살아도 스쳐간 사랑아 흩어질 눈물아 가슴 속 깊이 새긴 그 이름만 남겨줘 눈물이 마르면 가슴도 굳어져 더 아프지 않게 나를 도와줘 다신 날 찾지마 부탁해'

In [ ]:
song_lyrics_df_2000.가사[450]

'손을 따 봐 내 맘에 꼭 얹힌 네가 내려 갈까봐 내 사랑도 죽은 피처럼 빠져나갈까봐 가슴이 아파 너무 아파 어떡해 어떡해 맘을 쓸수록 맘이 헤져 닮아가 이렇게도 깊었니 손 쓸 새 없이 퍼져가간 너 날 떠날 때 다 데려가지 내 안에 너까지 눈물이 나와 터져 나와 그리워 그리워 잊으려해도 잊어내지 못하니 온 세상이 다 멈추고 내 심장이 또 멎으면 널 그때라면 끝낼까 낫지 않아 난 어떡해도 낫지 않을 걸 알아 난 내안에 살아 그냥 살아 날 해쳐도 되니 내안에 살아 그냥 살아 나를 해쳐도 참을거야'

In [ ]:
get_similar_song(song_lyrics_df_2000, song_lyrics_df_2000['가수&제목'][590])

입력한 노래 제목은 "백지영"의 "사랑안해"


,가수,노래제목
568,이수영,꽃들은 지고
269,오션 (5TION),More Than Words
54,god,Friday Night
838,엄정화,D.I.S.C.O (Feat. TOP)
250,여행스케치,달팽이와 해바라기 (Men`s Ver.)


In [ ]:
song_lyrics_df_2000['가사'][590]

'그럴려고 그랬어 돌아가려고 너의 차가움엔 그래 다 이유 있었던거야 나를 만지는 너의 손길 없어진 이제야 깨닫게 되었어 네맘 떠나간 것을 설마하는 그런 미련 때문에 그래도 나는 나를 위로해 나 이제 이러는 내가 더 가여워 이제라도 널 지울꺼야 기억의 모두를 이제 다시 사랑안해 말하는 난 너와 같은 사람 다시 만날 수가 없어서 사랑할 수 없어서 바보처럼 사랑 안해 말하는 널 사랑한다 나를 잊길바래 나를 지워줘 바보처럼 몰랐어 너희 두 사람 아직 기억하려던 그건 그래 다 욕심이야 다짐했건만 매일 아침 눈을 떠 지나간 너에게 기도해 나를 잊지 말라고 제발 지금 내가 바라는 하나 내 얘길 너무 쉽게 하진마 차라리 나를 모른다고 말해줘 시간지나 알게 될꺼야 내 사랑의 가치를 이제 다시 사랑안해 말하는 난 너와 같은 사람 다시 만날 수가 없어서 사랑할 수 없어서 바보처럼 사랑안해 말하는 널 사랑한다 나를 잊길 바래 나를 지워줘 내가 없는 내가 아닌 그 자리에 사랑 채우지마 혹시 만날 수가 있다면 사랑 할 수 있다면 아프잖아 사랑한 널 지켜보며 사랑한단 그 말 한마디를 하지 못해서'

In [ ]:
song_lyrics_df_2000['가사'][568]

'그래도 내 곁을 떠나가야만 한다면 어디로 가는지는 말하지 말아요 그대의 마음도 편치는 않을테니 됐어요 걱정은 말아요 어차피 한번은 언젠가 한번은 가야 할 길이라고 생각해 볼게요 꽃들이 피면 돌아온다는 덧없는 약속에 천천히 시들어 가길 원하진 않을테죠 저녁 무렵 함께 걸었던 그 길을 걷듯이 그렇게 떠나요 소중한 사람을 남겨 두기라도 한 듯 뒤돌아 보는 것도 제발 그만 해요 오히려 상처는 깊어져 갈 뿐이니 됐어요 이제는 가세요 어차피 한번은 언젠가 한번은 가야 할 길이라고 생각해 볼게요 꽃들이 피면 돌아온다는 덧 없는 약속에 천천히 시들어 가길 원하진 않을테죠 저녁 무렵 함께 걸었던 그 길을 걷듯이 그렇게 떠나요'

In [ ]:
lda_model = LDA(n_components = 3, random_state = 721, max_iter = 2) # 최대 3개 주제 추출

In [ ]:
lda_top = lda_model.fit_transform(tfidf) # Tfidf 결과물을 lda로 변환

In [ ]:
terms = vectorizer.get_feature_names() # 위에서 지정한 max_features인 750개의 단어가 저장됨

get_topics(lda_model.components_,terms)

Topic 1: [('그대', 18.24), ('내가', 15.31), ('나를', 12.51), ('나의', 12.09), ('내게', 11.88)]
Topic 2: [('사랑해', 15.39), ('나의', 13.98), ('내가', 12.87), ('나를', 11.56), ('그대', 11.42)]
Topic 3: [('너를', 25.61), ('내가', 19.71), ('너의', 18.14), ('다시', 17.8), ('나를', 17.79)]


### 90년대 노래 부분과 동일한 이유로 주제분석이 잘 이루어지지 않았다.

In [ ]:
song_lyrics_df_2000.to_csv('2000_2009_song.csv', index = False, encoding = 'utf-8')

## 전체 노래목록에서 유사곡 추천

In [ ]:
df = pd.concat([song_lyrics_1990_df, song_lyrics_df_2000], axis = 0, ignore_index = True).drop_duplicates('가사')
df.index = range(df.shape[0])

In [ ]:
df

,가수,노래제목,가사,text,가수&제목
0,변진섭,희망사항,청바지가 잘 어울리는 여자 밥을 많이 먹어도 배 안나오는 여자 내 얘기가 재미 없어...,변진섭희망사항청바지가 잘 어울리는 여자 밥을 많이 먹어도 배 안나오는 여자 내 얘기...,변진섭 희망사항
1,김민우,사랑일뿐야,나를 어떻게 생각하냐고 너는 내게 묻지만 대답하기는 힘들어 너에게 이런 얘길 한다면...,김민우사랑일뿐야나를 어떻게 생각하냐고 너는 내게 묻지만 대답하기는 힘들어 너에게 이...,김민우 사랑일뿐야
2,햇빛촌,유리창엔 비,낮부터 내린 비는 이 저녁 유리창에 이슬만 뿌려놓고서 밤이 되면 더욱 커지는 시계소...,햇빛촌유리창엔 비낮부터 내린 비는 이 저녁 유리창에 이슬만 뿌려놓고서 밤이 되면 더...,햇빛촌 유리창엔 비
3,김현식,비오는 날 수채화,빗방울 떨어지는 그 거리에 서서 그대 숨소리 살아있는 듯 느껴지면 깨끗한 붓 하나를...,김현식비오는 날 수채화빗방울 떨어지는 그 거리에 서서 그대 숨소리 살아있는 듯 느껴...,김현식 비오는 날 수채화
4,이승철,마지막 콘서트,지금 슬픈 내 모습은 무대뒤 한 소녀 애써 눈물 참으며 바라보고 있네 무대 뒤에 그...,이승철마지막 콘서트지금 슬픈 내 모습은 무대뒤 한 소녀 애써 눈물 참으며 바라보고 ...,이승철 마지막 콘서트
...,...,...,...,...,...
1807,손담비,아몰레드 (Amoled),빛나지 그 누구보다 더 밝게 더욱 빛나지 그 누가 발하는 빛보다 더 진하지 비교 조...,손담비아몰레드 (Amoled)빛나지 그 누구보다 더 밝게 더욱 빛나지 그 누가 발하...,손담비 아몰레드 (Amoled)
1808,김종국,행복병 (Feat. Shorry J (MM)),어떻게 내가 널 찾게 됐는지 내가 참 착하게 살아왔는지 누구나 한번은 행운이 오나봐...,김종국행복병 (Feat. Shorry J (MM))어떻게 내가 널 찾게 됐는지 내가...,김종국 행복병 (Feat. Shorry J (MM))
1809,마이티 마우스,연애특강 (Feat. 한예슬),수천 번을 말해도 늘 모자라는 말 자꾸 자꾸 들어도 듣고 싶은 말 마법처럼 우릴 설...,마이티 마우스연애특강 (Feat. 한예슬)수천 번을 말해도 늘 모자라는 말 자꾸 자...,마이티 마우스 연애특강 (Feat. 한예슬)
1810,유키스,만만하니,내가 그렇게렇게 만만하니 사랑이 그렇게 넌 만만하니 나와의 추억이 넌 만만하니 그렇...,유키스만만하니내가 그렇게렇게 만만하니 사랑이 그렇게 넌 만만하니 나와의 추억이 넌 ...,유키스 만만하니


### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features = 500) # 최다 빈출 500개 단어 사용
tfidf = vectorizer.fit_transform(df.가사)

cosine_sim = linear_kernel(tfidf, tfidf)

In [ ]:
indices = pd.Series(df.index, index = df['가수&제목']).drop_duplicates()

### 유사곡 리스트(top-5) 출력 함수

In [ ]:
get_similar_song(df, df['가수&제목'][1592])

In [ ]:
df.가사[1592]

In [ ]:
df.가사[608]

In [ ]:
df.to_csv('melon_1990_2009.csv', index = False, encoding = 'utf-8')

## Mecab 토큰화 후 유사도 계산


In [ ]:
cd /content/gdrive/My Drive/NIA

/content/gdrive/My Drive/NIA


In [ ]:
from konlpy.tag import Mecab

In [ ]:
mecab = Mecab()

In [ ]:
df = pd.read_csv('melon_1990_2009.csv')

### 명사만 추출

In [ ]:
token = []

text = df.가사

for i in range(len(text)):
  tk_text = mecab.nouns(text[i]) # 토큰화
  token.append(tk_text)

In [ ]:
df['가사토큰'] = token

In [ ]:
df.가사토큰 = [' '.join(x) for x in df.가사토큰]

In [ ]:
vectorizer = TfidfVectorizer(max_features = 500) # 최다 빈출 500개 단어 사용
tfidf = vectorizer.fit_transform(df.가사토큰)

In [ ]:
cosine_sim = linear_kernel(tfidf, tfidf)

In [ ]:
indices = pd.Series(df.index, index = df['가수&제목']).drop_duplicates()

In [ ]:
get_similar_song(df, df['가수&제목'][750])

입력한 노래 제목은 "김경호"의 "금지된 사랑"


,가수,노래제목
226,박정운,먼 훗날
1142,브라운 아이드 소울,정말 사랑했을까
856,컨츄리 꼬꼬,오! 가니
673,김경호,나의 사랑 천상(天上)에서도
761,핑클 (Fin.K.L),영원한 사랑


In [ ]:
df.가사[750]

'울지 마 여기에 새겨진 우리 이름을 봐 소중한 초대장이 젖어버리잖아 슬퍼마 너의 가족들이 보이지 않아도 언젠간 용서할 그 날이 올 거야 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속 해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 각오해 내게 무릎 꿇은 세상의 복수를 많은 시련 준대도 널 위해 견딜게 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 긴 세월 흐른 뒤 돌아보아도 아무런 후회 없도록 단 하룰 살아도 너 행복 하도록 만들 거야 내 사랑에 세상도 양보한 널 나 끝까지 아끼며 사랑할게 약속해줘 서로만 바라보다 먼 훗날 우리 같은 날에 떠나 사랑할게'

In [ ]:
df.가사[226]

'잊으려 잊으려 해도 잊을 수 없는 그 얼굴 지우려 지우려 해도 지울 수 없는 그 이름 이별을 아쉬워하며 나의 품에 안겨 고개 숙인 너 가슴 속 깊이 간직한 네게 하고 싶었던 그 한마디 남겨둔 채 돌아서는 슬픈 내 모습 뒤로 울먹이는 너를 느끼며 먼 훗날에 너를 다시 만나면 사랑했다 말을 할거야 때로는 외로움 속에 때로는 아쉬움 속에 세월이 흐른 어느 날 난 홀로 울고 말았지 이별을 아쉬워하며 나의 품에 안겨 흐느끼는 너 마음속 깊이 간직한 네게 하고 싶었던 그 한마디 말 못하고 돌아서는 슬픈 내 모습 뒤로 울먹이는 너를 느끼며 먼 훗날에 너를 다시 만나면 사랑했다 말을 할거야 돌아서는 슬픈 내 모습 뒤로 울먹이는 너를 느끼며 너는 나의 마음 알고 있을까 너를 진정 사랑했다고 먼 훗날에 너를 다시 만나면 사랑했다 말을 할거야'

In [ ]:
get_similar_song(df, df['가수&제목'][1598])

입력한 노래 제목은 "버즈"의 "사랑은 가슴이 시킨다 Part.2"


,가수,노래제목
1251,바이브,오래오래
733,엔알지 (NRG),사랑만들기
1037,서문탁,사미인곡
1291,리쌍,사랑은 (Song 정인)
1711,언터쳐블,It`s Okay (Feat. 화요비)


In [ ]:
df.가사[1598]

'그래 그렇게 걸어가면 돼 네 뒤에 선 날 두고서 나의 눈물이 들리지 않도록 너의 두 귀를 가리고서 남자라는 이유로 맘 편히 울지 못 하고 잘 가라는 손짓을 하지만 나의 가슴은 널 붙잡으라고 시키는데 내 사랑아 그대로 가 나를 떠나서 저 먼 곳에서 행복해 줘 나의 사랑은 잊어줘 모두 다 나의 곁에서 잠이 들었던 흔적까지 지워줘 많이 아파도 웃어내 볼게 남자답게 보내줄게 내가 해주는 마지막 선물은 이별이란 한 가지 뿐 네가 하고 싶은 것 늘 네가 갖고 싶은 것 어느 하나 해준 게 없는데 무슨 염치로 날 떠나가는 널 붙잡겠니 내 사랑아 그대로가 나를 떠나서 저 먼 곳에서 행복해 줘 나의 사랑은 잊어줘 모두 다 나의 곁에서 잠이 들었던 흔적까지 너의 뒷 모습이 어색해 보여도 내 눈 가득히 담아둘게 내 사랑아 널 보낼게 내 맘 변해서 너를 잡으면 안되니까 우리 사랑은 여기까진 가봐 너와 함께한 시간 모두다 지울게 널 사랑해'

In [ ]:
df.가사[1251]

'난 이별을 몰라서 사랑만 알아서 많이 힘이 든가봐 넌 이별을 알아서 사랑은 몰라서 많이 쉬웠었나봐 너를 사랑하는 한 남자로 태어나서 나는 단 한번도 후회한적 없었다고 나 너를 위해 목숨까지 다 바쳐서 사랑했지만 이젠 나에겐 눈물이 되는걸 부디 오래오래 오래오래 오래오래 오래오래 그대곁에 있을께 심장이 멈출때 까지 기쁜 날들을 행복을 줄게 오래오래 오래오래 오래오래 오래오래 그냥 곁에 있을께 시간이 멈출때 까지 슬픈날들을 우리는 없게 넌 모를꺼야 내가 얼마만큼 너를 사랑하는지 너를 사랑하는 한 남자로 태어나서 나는 단 한번도 후회한적 없었다고 나 너를 위해 목숨까지 다 바쳐서 사랑했지만 이젠 나에겐 눈물이 되는걸 부디 오래오래 오래오래 오래오래 오래오래 다신 보지 말자고 다신 볼순 없다고 오래오래 오래오래 오래오래 오래오래 너를 잊고살아도 그냥 곁에있을께 다신 볼수없어도 내가 숨을 쉴때까진 너에 곁에 있을께 내가 널 지켜줄게 너를 사랑하는 한 남자로 태어나서 너의 곁에 있을께 내가 널 지켜줄게 너를 사랑하는 남자로 태어나서 나는 단 한번도 후회한적 없었다고 너를 위해 지옥까지 갈 수 있는 나의 사랑아 내가 너의 천사가 되줄게'

In [ ]:
get_similar_song(df, df['가수&제목'][1738])

입력한 노래 제목은 "4minute"의 "Hot Issue"


,가수,노래제목
965,드렁큰 타이거,Good Life
1715,크라운제이 (CROWN J),Fly Boy
825,김연우,거짓말 같은 시간
791,김종서,실연
1467,SG 워너비,폭풍


In [ ]:
df.가사[1738]

'머리부터 발 끝까지 내 모든 것 하나하나 모두 다 모두 다 난 항상 생각 없이는 내가 내가 신은 슈즈 내 맘대로 또 자꾸자꾸 하는 포즈 아무렇게나 살짝살짝 바른 루즈 내 스타일 하나하나 모두 다 부럽니 그렇게 어설프게 따라할 순 없지 원하면 따라와봐 내 스타일 따라해봐 난 항상 내가 내가 보는 것들까지 내가 내가 듣는 것들까지 내가 내가 말하는 것까지 머리부터 발 끝까지 내 모든 것 하나하나 모두 다 모두 다 난 항상 난 항상 또또또 쳐다보지 내게 모여들지 날 보고 소리치지 멈추지 않는 나처럼 하고 싶니 나처럼 되고 싶니 날 따라 움직여봐 내가 나의 눈빛 하나까지 내가 나의 손짓 하나까지 내가 내가 춤추는 것까지 머리부터 발 끝까지 내 모든 것 하나하나 모두 다 모두 다 난 항상 너보다 잘록한 허리 쫙 빠져 미끈한 다리 누구보다 당당한 난 항상 머리부터 발 끝까지 내 모든 것 하나하나 모두 다 모두 다 난 항상 난 항상'

In [ ]:
df.가사[965]

'한잔 두잔 비워내는 술잔 혀를 지나 목구멍으로 넘어가는 순간 이제부터 시작되는 너와나의 시간 흘러가는 나그네들에 유일한 위안 한잔의 데낄라 같은 쓰디쓴 내 인생은 세상 의미 현실에 우물안 같은 내 버림 내 삶 깊이 너와나 바꿔 삶에 그림자 같은 의미 끝없는 경쟁 속에 나는 우정속에 터지는 기쁨에 눈물 조이는 내 목에 텅 빈 내맘 위해 불러보는 슬픈노래 손에 슬픈 술잔 속에 친구 자니 힙합은 나의 술 이젠 내 가슴에 파인 상처로 매꿔 그리고 마셔 여지껏 쌓인 나의 울분을 토해 뱉어 채워줄께 가득히 잔을위로 머리위로 건배 세상 우리위한 축배를 잔을위로 머리위로 건배 세상 조용히 히 해 내 말좀 들어봐봐 내 술잔처럼 깨져버린 내인생의 나를 버리고 간 너 곧 후회하지 지 지금 누구도 나를 건든다면 후회하지 히 나는 주정뱅이 끝없이 지껄이는 나 비틀비틀거리는 쓰러져가는 팽이 괜히 저 눈길이 가는 저 에 항상 내 기대치만큼 더 이상에 만큼 좀 더 깊이 다가가지 못할지라도 항상 텅빈 나의 가슴 채우지 못할지라도 내 인생과 난모든 진실을 위해 내 인생 그것을 위해 우리들에 입에 독한물에 취해 나에 심장으로 부터나온 삶에 많은 얘기 후에 와 같이 바람과 함께 나는 사라지네 채워줄께 가득히 잔을위로 머리위로 건배 세상 우리위한 축배를 잔을위로 머리위로 건배 세상 채워줄께 가득히 잔을위로 머리위로 건배 세상 우리위한 축배를 잔을위로 머리위로 건배 세상 채워줄께 가득히 잔을위로 머리위로 건배 세상 우리위한 축배를 잔을위로 머리위로 건배 세상'

### LDA 적용


In [ ]:
lda_model = LDA(n_components = 7, random_state = 721, max_iter = 2) # 최대 7개 주제 추출

In [ ]:
lda_top = lda_model.fit_transform(tfidf) # Tfidf 결과물을 lda로 변환

In [ ]:
terms = vectorizer.get_feature_names() # 위에서 지정한 max_features인 500개의 단어가 저장됨
get_topics(lda_model.components_,terms)

Topic 1: [('사랑', 13.46), ('나나', 11.52), ('그대', 10.21), ('세상', 9.72), ('우리', 7.6)]
Topic 2: [('사랑', 24.3), ('사람', 11.78), ('우리', 11.53), ('이별', 9.76), ('마음', 9.68)]
Topic 3: [('그대', 34.5), ('그녀', 31.64), ('사랑', 24.76), ('세상', 15.71), ('우리', 15.45)]
Topic 4: [('사랑', 60.79), ('그대', 35.24), ('마음', 23.24), ('가슴', 17.19), ('눈물', 16.02)]
Topic 5: [('사랑', 79.89), ('그대', 67.16), ('그댈', 25.87), ('사람', 24.55), ('눈물', 21.13)]
Topic 6: [('그대', 31.21), ('사랑', 24.97), ('세상', 13.05), ('사람', 12.75), ('모습', 12.0)]
Topic 7: [('사랑', 52.41), ('그대', 29.99), ('사람', 24.9), ('생각', 22.81), ('모습', 20.95)]
